<a href="https://colab.research.google.com/github/lucasrigobello/GeminAi_English_Trainer/blob/main/GeminAi_English_Trainer_App.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#GeminAi English Trainer

Welcome to our cutting-edge app designed to prepare you for English proficiency exams like never before.

<img src="https://github.com/lucasrigobello/GeminAi_English_Trainer/blob/2f1ac8074a47a4d2e52fa3f66195d39a2de527ac/GeminAiEnglishTrainer.png?raw=true" width="250">

The GeminAi English Trainer App uses the Gemini generative artificial intelligence for teaching and learning English. Harnessing the power of generative artificial intelligence, we offer a comprehensive platform for targeted training and exam readiness. Whether you're aiming for certifications or seeking to enhance your language skills, our app provides a tailored learning experience that adapts to your needs and accelerates your progress. With interactive exercises, personalized feedback, and simulated exam environments, we empower you to confidently master the language and excel in your proficiency goals.

Get ready to embark on a journey of language mastery like never before. Welcome to GeminAi English Trainer.



## **Rode cada célula de código** (botão play a esquerda de cada célula)

# Preparando o Aplicativo


In [ ]:
# install the packages needed
!pip install ffmpeg-python

In [ ]:
# Importing the packages
from IPython.display import Markdown
import random
import gdown

# pagakes for the audio tasks
from IPython.display import HTML, Audio
from google.colab.output import eval_js
from base64 import b64decode
import numpy as np
import scipy
from scipy.io.wavfile import read as wav_read
import io
import ffmpeg

# Google Gemini packages
import google.generativeai as genai
from google.colab import userdata
genai.configure(api_key = userdata.get('secretApiKey'))

In [ ]:
# The interface to audio recording

AUDIO_HTML = """
<script>
var my_div = document.createElement("DIV");
var my_p = document.createElement("P");
var my_btn = document.createElement("BUTTON");
var t = document.createTextNode("Press to start recording");

my_btn.appendChild(t);
//my_p.appendChild(my_btn);
my_div.appendChild(my_btn);
document.body.appendChild(my_div);

var base64data = 0;
var reader;
var recorder, gumStream;
var recordButton = my_btn;

var handleSuccess = function(stream) {
  gumStream = stream;
  var options = {
    //bitsPerSecond: 8000, //chrome seems to ignore, always 48k
    mimeType : 'audio/webm;codecs=opus'
    //mimeType : 'audio/webm;codecs=pcm'
  };
  //recorder = new MediaRecorder(stream, options);
  recorder = new MediaRecorder(stream);
  recorder.ondataavailable = function(e) {
    var url = URL.createObjectURL(e.data);
    var preview = document.createElement('audio');
    preview.controls = true;
    preview.src = url;
    document.body.appendChild(preview);

    reader = new FileReader();
    reader.readAsDataURL(e.data);
    reader.onloadend = function() {
      base64data = reader.result;
      //console.log("Inside FileReader:" + base64data);
    }
  };
  recorder.start();
  };

recordButton.innerText = "Recording... press to stop";

navigator.mediaDevices.getUserMedia({audio: true}).then(handleSuccess);


function toggleRecording() {
  if (recorder && recorder.state == "recording") {
      recorder.stop();
      gumStream.getAudioTracks()[0].stop();
      recordButton.innerText = "Saving the recording... pls wait!"
  }
}

// https://stackoverflow.com/a/951057
function sleep(ms) {
  return new Promise(resolve => setTimeout(resolve, ms));
}

var data = new Promise(resolve=>{
//recordButton.addEventListener("click", toggleRecording);
recordButton.onclick = ()=>{
toggleRecording()

sleep(2000).then(() => {
  // wait 2000ms for the data to be available...
  // ideally this should use something like await...
  //console.log("Inside data:" + base64data)
  resolve(base64data.toString())

});

}
});

</script>
"""

In [ ]:
# Audio recording function

def get_audio():
  display(HTML(AUDIO_HTML))
  data = eval_js("data")
  binary = b64decode(data.split(',')[1])

  process = (ffmpeg
    .input('pipe:0')
    .output('pipe:1', format='wav')
    .run_async(pipe_stdin=True, pipe_stdout=True, pipe_stderr=True, quiet=True, overwrite_output=True)
  )
  output, err = process.communicate(input=binary)

  riff_chunk_size = len(output) - 8
  # Break up the chunk size into four bytes, held in b.
  q = riff_chunk_size
  b = []
  for i in range(4):
      q, r = divmod(q, 256)
      b.append(r)

  # Replace bytes 4:8 in proc.stdout with the actual size of the RIFF chunk.
  riff = output[:4] + bytes(b) + output[8:]

  sr, audio = wav_read(io.BytesIO(riff))

  return audio, sr

In [ ]:
database_dict = {
    'Track 1.mp3': '1goSZwV8clvpoRtMYi26MLT1mNrJiaPrS',
    'Track 2.mp3': '1VB84t2zhyMhHMvH94a59aEVokOZQte3A',
    'Track 3.mp3': '1RISbzsE1Rq1uhtlAU7owiYhV1uKn2Fq3',
    'Track 4.mp3': '1G5kCttzbFvX9V9i_0LKxPjjSphhiM0YF',
    'Track 5.mp3': '14KYMWak8TD5Reh8TIDXBjz6Ys2jjhDXd',
    'Track 6.mp3': '1sEfiwIEaUwRDlqElmHtVfpFJ-aRy_10x',
    'Track 7.mp3': '1yg9hH7J9nyhhc_TkmnmCbqSvrEvDH9Ta',
    'Track 8.mp3': '1tpthz4XF0WkM7YRhx2Yi-pmbOR_W39WK',
    'Track 9.mp3': '1T_unacLuzoZRY7OMwpCmILfJ4g_tb_rJ',
    'Track 10.mp3': '1mC9HhiKq0sUHIcHLGVx7jAba2Xt05eet',
    'Track 11.mp3': '1MVJnKjwKUMW4Ah93_fcYFybnTmOXlzKi',
    'Track 12.mp3': '1f1NRIe4J6B4iuRdOzpohC9iS6P3c5qY0',
    'Track 13.mp3': '1MzJkmH9dTeKEHHVF3JW91njWxgLyWiO0',
    }


# Escute o áudio a seguir

In [ ]:
# Selecting a random track option from the database
list_number = list(range(1,14))
sample_selection = f'Track {random.choice(list_number)}.mp3'

# Downlaoding audio track from database on google Drive to the Colab
gdown.download(id = database_dict[sample_selection],
               output = sample_selection
               )

In [ ]:
# Play the audio
display(Markdown("## Aperte play para escutar o áudio"))
Audio(sample_selection)

# Grave a sua resposta
### Explique as ideias presentes no áudio que você acabou de ouvir
### Atente-se as questões de pronúncia e gramática

In [ ]:
# Ao rodar essa célula, a gravação se inicia automaticamente.
# Aperte o botão para pausar e encerrar, assim que finalizar sua resposta.
# E aguarde que o seu áudio seja salvo.

display(Markdown("# Gravação já está ativa!"))
display(Markdown("### Aperte o botão para pausar e encerrar, assim que finalizar sua resposta"))
audio, sr = get_audio()
scipy.io.wavfile.write('recording.wav', sr, audio)

# Feedback da **GeminAi English Trainer**
### Observe as análises e susgestões providas pela GeminAi English Trainer

*   Coerência com o texto original
*   Sugestões de correções de pronúncia
*   Sugestões de correções gramaticais





In [ ]:
# Preparing the Generative Model Gemini 1.5
model = genai.GenerativeModel('gemini-1.5-pro-latest')
chat = model.start_chat(history=[])

## If it gets an error, please run once again
# Upload the Sample Audio file
sample_file = genai.upload_file(path=sample_selection, display_name="Sample_audio")
# Upload the Answer Audio file
recording = genai.upload_file(path="recording.wav", display_name="Answer audio")

# First Context to the Gemini Ai
context = ["Contextualize and give essential information based on the following audio: ",
           sample_file]

# Get the Gemini Ai respose
response = chat.send_message(context)
display(Markdown('## GeminAi English Trainer interpretation:'))
Markdown(response.text)

In [ ]:
# Context to the Gemini Ai on examplained recording
context = ['You are an English proficiency test evaluator. Based on the last conversation, you should analyse how well the ideas and information are explained in the following respose: ',
           recording]

# Get the Gemini Ai respose
response = chat.send_message(context)
display(Markdown('## GeminAi English Trainer response evaluation:'))
Markdown(response.text)

In [ ]:
# Context to the Gemini Ai to give feedback on pronunciation
context = ['You are still an English proficiency test evaluator, make some appointments and suggestions to improve pronunciation for the foreign english learner, based on the following respose: ',
           recording]

# Get the Gemini Ai respose
response = chat.send_message(context)
display(Markdown('## GeminAi English Trainer pronunciation suggestions:'))
Markdown(response.text)

In [ ]:
# Context to the Gemini Ai to give feedback on grammar
context = ['Now, you are still an English proficiency test evaluator, make grammar corretions on the following speech: ',
           recording]

# Get the Gemini Ai respose
response = chat.send_message(context)
display(Markdown('## GeminAi English Trainer grammar suggestions:'))
Markdown(response.text)